In [1]:
# pip install nltk tensorflow
# nltk.download('punkt')
# nltk.download('wordnet')

In [12]:
import nltk
import json
import pickle
import numpy as np
import random

from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import tensorflow as tf

In [2]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '/', '@', '$']

In [3]:
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [15]:
# Data Preprocessing 
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        documents.append((w, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [6]:
# Data converted to bag of words
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = [0] * len(words)
    
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in pattern_words:
        if w in words:
            bag[words.index(w)] = 1
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

random.shuffle(training)

In [8]:
train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])

In [16]:
# Training the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [13]:
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [14]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs= 200, batch_size = 5, verbose=1)
model.save('chatbot_model_2.h5', hist)
print("model created")

Epoch 1/200
25/25 [==============================] - 1s 1ms/step - loss: 3.2020 - accuracy: 0.0403 
Epoch 2/200
25/25 [==============================] - 0s 959us/step - loss: 3.1553 - accuracy: 0.0806
Epoch 3/200
25/25 [==============================] - 0s 959us/step - loss: 3.0520 - accuracy: 0.1048
Epoch 4/200
25/25 [==============================] - 0s 1ms/step - loss: 3.0032 - accuracy: 0.1210
Epoch 5/200
25/25 [==============================] - 0s 959us/step - loss: 2.9231 - accuracy: 0.1371
Epoch 6/200
25/25 [==============================] - 0s 834us/step - loss: 2.8560 - accuracy: 0.1452
Epoch 7/200
25/25 [==============================] - 0s 1ms/step - loss: 2.7901 - accuracy: 0.1694
Epoch 8/200
25/25 [==============================] - 0s 833us/step - loss: 2.6878 - accuracy: 0.2339
Epoch 9/200
25/25 [==============================] - 0s 875us/step - loss: 2.5876 - accuracy: 0.2581
Epoch 10/200
25/25 [==============================] - 0s 959us/step - loss: 2.4067 - accuracy: 0

e:\Edu_Chatbot\dumbChat(2)\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
